# Import libraries

In [1]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from PIL import Image

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

2.6.4
Found GPU at: /device:GPU:0


2022-11-14 15:51:13.984017: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-14 15:51:14.050370: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 15:51:14.150966: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 15:51:14.151778: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [2]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [3]:
print("--> Checking for physical Tensorflow devices")
for device in tf.config.list_physical_devices():
    print(": {}".format(device.name))

--> Checking for physical Tensorflow devices
: /physical_device:CPU:0
: /physical_device:GPU:0


2022-11-14 15:51:16.589724: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 15:51:16.590551: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 15:51:16.591188: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


# Load data

Solo questo dovrebbe essere diverso durante la challenge

In [4]:
dataset_dir = '/kaggle/input/homework1/training_data_final'
image_size = (96,96)

# Dataloader and Data Augmentation

We can fine tune this

In [5]:
noaug_train_data_gen = tfk.preprocessing.image.ImageDataGenerator()

aug_train_data_gen = tfk.preprocessing.image.ImageDataGenerator(rotation_range=30,
                                        height_shift_range=50,
                                        width_shift_range=50,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True, 
                                        fill_mode='reflect',
                                        validation_split=0.2)

aug_validation_data_gen = tfk.preprocessing.image.ImageDataGenerator(
                                        validation_split=0.2)

train_gen = aug_train_data_gen.flow_from_directory(directory=dataset_dir,
                                               target_size=image_size,
                                               color_mode='rgb',
                                               class_mode='categorical',
                                               batch_size=8,
                                               shuffle=True,
                                               seed=seed,
                                               subset='training')

train_gen_all = noaug_train_data_gen.flow_from_directory(directory=dataset_dir,
                                               target_size=image_size,
                                               color_mode='rgb',
                                               class_mode='categorical',
                                               batch_size=8,
                                               seed=seed)

validation_gen = aug_validation_data_gen.flow_from_directory(directory=dataset_dir,
                                               target_size=image_size,
                                               color_mode='rgb',
                                               class_mode='categorical',
                                               batch_size=8,
                                               shuffle=True,
                                               seed=seed,
                                               subset='validation')

Found 2836 images belonging to 8 classes.
Found 3542 images belonging to 8 classes.
Found 706 images belonging to 8 classes.


In [6]:
print("Assigned labels")
print(train_gen.class_indices)
print()
print("Target classes")
print(train_gen.classes)
num_classes = train_gen.num_classes

Assigned labels
{'Species1': 0, 'Species2': 1, 'Species3': 2, 'Species4': 3, 'Species5': 4, 'Species6': 5, 'Species7': 6, 'Species8': 7}

Target classes
[0 0 0 ... 7 7 7]


# Load pre-trained model for transfer learning

We can fine tune this with different pretrained models

In [7]:
# Create the base model from the pre-trained model
IMG_SHAPE = image_size + (3,)
base_model = tf.keras.applications.EfficientNetB2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
model_name = 'EfficientNetB2'

2022-11-14 15:51:17.601373: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 15:51:17.602390: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 15:51:17.603127: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 15:51:17.603847: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 15:51:17.604511: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

31801344/31790344 [==============================] - 0s 0us/step


In [8]:
base_model.summary()

Model: "efficientnetb2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 96, 96, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 96, 96, 3)    7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 97, 97, 3)    0           normalization[0][0]              
_____________________________________________________________________________________

In [9]:
def build_model(input_shape, output_classes, learning_rate=0.001, freeze=True):
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')

    # Freeze the base model
    if(freeze):
      base_model.trainable = False
    else:
      base_model.trainable = True
    # We need training=False for the BatchNormalization layer
    feature_extractor = base_model(input_layer , training=False)

    x = tfkl.GlobalAveragePooling2D()(feature_extractor)
    x = tfkl.Dropout(0.1)(x)
    x = tfkl.Dense(units=256, activation='relu', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='hidden_layer_1')(x)
    x = tfkl.Dropout(0.2)(x)
    output_layer = tfkl.Dense(units=output_classes, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='output_layer')(x)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=learning_rate), metrics='accuracy')

    # Return the model
    return model

def build_custom_model(input_shape, output_classes, learning_rate=0.001):
    chanDim = -1
    levels = 4
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')
    x = tfkl.Conv2D(32, (7, 7), padding="same")(input_layer)
    x = tfkl.LeakyReLU(alpha=0.3)(x)
    x = tfkl.BatchNormalization(axis=chanDim)(x)
    x = tfkl.Add([input_layer,x])
    # CNN 
    for i in range(levels):
        first = tfkl.MaxPooling2D(pool_size=(3, 3))(x)
        x = tfkl.Conv2D(32*(i+2), (3, 3), padding="same")(first)
        x = tfkl.LeakyReLU(alpha=0.3)(x)
        x = tfkl.BatchNormalization(axis=chanDim)(x)
        x = tfkl.Add([first,x])
    x = tfkl.MaxPooling2D(pool_size=(3, 3))(x)
    
    # FC
    x = tfkl.GlobalAveragePooling2D()(x)
    x = tfkl.Dropout(0.2)(x)
    x = tfkl.Dense(units=256, activation='relu', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='hidden_layer_1')(x)
    x = tfkl.Dropout(0.3)(x)
    output_layer = tfkl.Dense(units=output_classes, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='output_layer')(x)
    
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')
              
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=learning_rate),metrics="accuracy")
    return model

In [10]:
saved_as_dataset = False
if(saved_as_dataset):
    model = tfk.models.load_model(f'/kaggle/input/efficientnetb2/{model_name}.h5')   
else:
    model = build_model(IMG_SHAPE, num_classes, freeze=True)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 96, 96, 3)]       0         
_________________________________________________________________
efficientnetb2 (Functional)  (None, 3, 3, 1408)        7768569   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1408)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1408)              0         
_________________________________________________________________
hidden_layer_1 (Dense)       (None, 256)               360704    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
output_layer (Dense)         (None, 8)                 2056  

In [11]:
train_FC = True
if(train_FC):
    history = model.fit(
            x = train_gen,
            epochs = 50,
            validation_data = validation_gen
        ).history
    model.save(f"./models/{model_name}_FC_only.h5")

2022-11-14 15:51:21.982259: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50


2022-11-14 15:51:28.998594: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


355/355 [==============================] - 49s 102ms/step - loss: 1.4666 - accuracy: 0.4640 - val_loss: 1.1456 - val_accuracy: 0.5779
Epoch 2/50
355/355 [==============================] - 14s 39ms/step - loss: 1.2295 - accuracy: 0.5628 - val_loss: 1.0688 - val_accuracy: 0.6091
Epoch 3/50
355/355 [==============================] - 15s 43ms/step - loss: 1.1882 - accuracy: 0.5621 - val_loss: 1.0466 - val_accuracy: 0.6048
Epoch 4/50
355/355 [==============================] - 14s 40ms/step - loss: 1.1377 - accuracy: 0.5864 - val_loss: 1.0583 - val_accuracy: 0.6076
Epoch 5/50
355/355 [==============================] - 15s 42ms/step - loss: 1.1015 - accuracy: 0.5906 - val_loss: 0.9967 - val_accuracy: 0.6317
Epoch 6/50
355/355 [==============================] - 15s 42ms/step - loss: 1.0834 - accuracy: 0.6181 - val_loss: 0.9866 - val_accuracy: 0.6601
Epoch 7/50
355/355 [==============================] - 14s 39ms/step - loss: 1.0504 - accuracy: 0.6266 - val_loss: 0.9213 - val_accuracy: 0.6629
Ep

/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [12]:
# Fine tune also the last 5 layers of the feature extraction part
train_CNN = True
model = tfk.models.load_model(f"./models/{model_name}_FC_only.h5")
if(train_CNN):
    leave_freeze = len(model.get_layer('efficientnetb2').layers) - 5
    for layer in model.get_layer('efficientnetb2').layers[:leave_freeze]:
       layer.trainable = False
    for layer in model.get_layer('efficientnetb2').layers[leave_freeze:]:
       layer.trainable = True
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(),metrics="accuracy")
    model.summary()
    history = model.fit(
            x = train_gen,
            epochs = 30,
            validation_data = validation_gen
        ).history
    model.save(f"./models/{model_name}_CNN_tuned.h5")

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 96, 96, 3)]       0         
_________________________________________________________________
efficientnetb2 (Functional)  (None, 3, 3, 1408)        7768569   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1408)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1408)              0         
_________________________________________________________________
hidden_layer_1 (Dense)       (None, 256)               360704    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
output_layer (Dense)         (None, 8)                 2056  

In [13]:
# Fine tune all the model
train_all = True
model = tfk.models.load_model(f"./models/{model_name}_CNN_tuned.h5")
if(train_all):
    for layer in model.get_layer('efficientnetb2').layers:
       layer.trainable = True
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=3e-4),metrics="accuracy")
    model.summary()
    history = model.fit(
            x = train_gen,
            epochs = 100,
            validation_data = validation_gen
        ).history
    model.save(f"./models/{model_name}_all.h5")

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 96, 96, 3)]       0         
_________________________________________________________________
efficientnetb2 (Functional)  (None, 3, 3, 1408)        7768569   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1408)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1408)              0         
_________________________________________________________________
hidden_layer_1 (Dense)       (None, 256)               360704    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
output_layer (Dense)         (None, 8)                 2056  

In [14]:
# Retrain with all dataset very low learining rate
last_train = True
model = tfk.models.load_model(f"./models/{model_name}_all.h5")
if(last_train):
    for layer in model.get_layer('efficientnetb2').layers:
       layer.trainable = True
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.SGD(learning_rate=0.000001),metrics="accuracy")
    model.summary()
    history = model.fit(
                x = train_gen_all,
                epochs = 10
            ).history
    model.save(f"./models/{model_name}_final.h5")

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 96, 96, 3)]       0         
_________________________________________________________________
efficientnetb2 (Functional)  (None, 3, 3, 1408)        7768569   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1408)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1408)              0         
_________________________________________________________________
hidden_layer_1 (Dense)       (None, 256)               360704    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
output_layer (Dense)         (None, 8)                 2056  

# Test subission

In [15]:
class Model:
    def __init__(self, path):
        self.model = tf.keras.models.load_model(os.path.join(path, f"{model_name}_final.h5"))

    def predict(self, X):
        out = self.model.predict(X)
        out = tf.argmax(out, axis=-1)
        return out
test_model = Model("./models")

In [16]:
image = Image.open('/kaggle/input/homework1/training_data_final/Species6/00042.jpg')
frame = np.expand_dims(np.asarray(image), 0)
test_model.predict(frame)

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([5])>